In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 80% !important; }</style>"))

In [1]:
import sys
import time
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import colors

In [ ]:
myColors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
            '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
            '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000',
            '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', 
            '#307D7E', '#000000', "#DDEFFF", "#000035", "#7B4F4B", 
            "#A1C299", "#300018", "#C2FF99", "#0AA6D8", "#013349", 
            "#00846F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", 
            "#1E6E00", "#DFFB71", "#868E7E", "#513A01", "#CCAA35"]

colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7,0.8,20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

In [2]:
sys.path.append("../../")
from SMaSH import SMaSH

sm = SMaSH()

 * Initialising ...



In [5]:
help(sm.remove_features_pct)

Help on method remove_features_pct in module SMaSH.SMaSH:

remove_features_pct(adata, group_by=None, pct=0.3) method of SMaSH.SMaSH.SMaSH instance
    Removing ... from AnnData object.
    
    Parameters
    ----------
    adata : anndata.AnnData
            The AnnData matrix of shape `n_obs` × `n_vars`.
            Rows correspond to cells and columns to genes.
    group_by : string (default: None)
            ...
    pct : flaot (default: 0.3)
            ... 
    
    Returns
    -------
    adata : anndata.AnnData
            The AnnData without ... genes.



In [ ]:
sys.path.append("/home/ubuntu/Taneda/Functions")
from scRNA_functions import scRNA_functions

fc = scRNA_functions()

# Loading annData object

In [ ]:
obj = sc.datasets.paul15()

In [ ]:
print("%d genes across %s cells"%(obj.n_vars, obj.n_obs))

In [ ]:
dict_annotation = {}

dict_annotation['1Ery']='Ery'
dict_annotation['2Ery']='Ery'
dict_annotation['3Ery']='Ery'
dict_annotation['4Ery']='Ery'
dict_annotation['5Ery']='Ery'
dict_annotation['6Ery']='Ery'
dict_annotation['7MEP']='MEP'
dict_annotation['8Mk']='Mk'
dict_annotation['9GMP']='GMP'
dict_annotation['10GMP']='GMP'
dict_annotation['11DC']='DC'
dict_annotation['12Baso']='Baso'
dict_annotation['13Baso']='Baso'
dict_annotation['14Mo']='Mo'
dict_annotation['15Mo']='Mo'
dict_annotation['16Neu']='Neu'
dict_annotation['17Neu']='Neu'
dict_annotation['18Eos']='Eos'
dict_annotation['19Lymph']='Lymph'

annotation = []
for celltype in obj.obs["paul15_clusters"].tolist():
    annotation.append(dict_annotation[celltype])

obj.obs['annotation'] = annotation
obj.obs['annotation'] = obj.obs['annotation'].astype("category")

#### Data preparation

In [ ]:
sf.data_preparation(obj)

#### Removing general genes

In [ ]:
s1 = time.time()

In [ ]:
obj = sf.remove_general_genes(obj)

#### Removing house-keeping genes

http://www.housekeeping.unicamp.br/?homePageGlobal

In [ ]:
obj = sf.remove_housekeepingenes(obj, path="../../data/house_keeping_genes_Mouse_bone_marrow.txt")
obj = sf.remove_housekeepingenes(obj, path="../../data/house_keeping_genes_Mouse_HSC.txt")

#### Removing genes expressed in less than 30% within groups

In [ ]:
obj = sf.remove_features_pct(obj, group_by="annotation", pct=0.3)

#### Removing genes expressed in more than 50% in a given group where genes are expressed for more 75% within a given group

In [ ]:
obj = sf.remove_features_pct_2groups(obj, group_by="annotation", pct1=0.75, pct2=0.5)

#### Revert PCA

In [ ]:
obj = sf.scale_filter_features(obj, n_components=None, filter_expression=True)

#### DNN

In [ ]:
s2 = time.time()

In [ ]:
sf.DNN(obj, group_by="annotation", model=None, balance=True, verbose=True, save=False)

#### Shap value

In [ ]:
selectedGenes, selectedGenes_dict = sf.run_shap(obj, group_by="annotation", model=None, verbose=True, pct=0.1, restrict_top=("local", 20))

In [ ]:
e2 = time.time()

#### Classifiers

In [ ]:
sf.run_classifiers(obj, group_by="annotation", genes=selectedGenes, classifier="KNN", balance=True, title="DNN-KNN", save=False)

#### Sorting genes per cluster

In [ ]:
axs, selectedGenes_top_dict = sf.sort_and_plot(obj, selectedGenes, group_by="annotation", group_by2=None, top=4, figsize=(6,10))

In [ ]:
e1 = time.time()

# Elapsed time

In [ ]:
print("%d genes across %s cells"%(obj.n_vars, obj.n_obs))

In [ ]:
print('Elapsed time (s): ', e1-s1)

In [ ]:
print('Elapsed time (s): ', e2-s2)